In [1]:
from transformers import CLIPProcessor, CLIPModel
import torch



# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [23]:
# Fixed list of clothing categories (your known types)
clothing_types = ['shirt, blouse', 'top, t-shirt, sweatshirt', 'sweater', 'cardigan', 'jacket', 'vest', 'pants', 'shorts', 'skirt', 'coat', 'dress', 'jumpsuit', 'cape', 'glasses', 'hat', 'headband, head covering, hair accessory', 'tie', 'glove', 'watch', 'belt', 'leg warmer', 'tights, stockings', 'sock', 'shoe', 'bag, wallet', 'scarf', 'umbrella', 'hood', 'collar', 'lapel', 'epaulette', 'sleeve', 'pocket', 'neckline', 'buckle', 'zipper', 'applique', 'bead', 'bow', 'flower', 'fringe', 'ribbon', 'rivet', 'ruffle', 'sequin', 'tassel']

In [3]:

def find_matching_clothing_types(user_text, types_list, top_k=3):
    # Encode user input text
    inputs_user = clip_processor(text=[user_text], return_tensors="pt", padding=True)
    user_emb = clip_model.get_text_features(**inputs_user)
    user_emb = user_emb / user_emb.norm(dim=-1, keepdim=True)  # normalize

    # Encode clothing type texts
    inputs_types = clip_processor(text=types_list, return_tensors="pt", padding=True)
    types_emb = clip_model.get_text_features(**inputs_types)
    types_emb = types_emb / types_emb.norm(dim=-1, keepdim=True)

    # Compute cosine similarity (user text vs each clothing type)
    similarities = (user_emb @ types_emb.T).squeeze(0)

    # Get top-k matches
    topk_sim, topk_idx = torch.topk(similarities, k=top_k)
    topk_types = [types_list[i] for i in topk_idx]

    # Return top matches with scores
    return list(zip(topk_types, topk_sim.detach().cpu().numpy()))


In [9]:

# Example usage
user_query = input("leather item ")
matches = find_matching_clothing_types(user_query, clothing_types)

print("Top clothing types matched to your query:")
for ctype, score in matches:
    print(f"{ctype} (score: {score:.3f})")


Top clothing types matched to your query:
coat (score: 1.000)
jacket (score: 0.921)
cape (score: 0.872)


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/gemma-3/pyTorch/gemma-3-4b-it")

print("Path to model files:", path)





100%|██████████| 4.47M/4.47M [00:02<00:00, 1.97MB/s]


























































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# model_path = '/home/abhi/.cache/kagglehub/models/google/gemma/transformers/1.1-2b-it/1'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [53]:
def get_top3_clothing_types(user_text, clothing_types, model, tokenizer):
    prompt = f"""You are a helpful fashion assistant.

        Given the following user request: "{user_text}"

        Choose and list the 3 most relevant clothing items from this list: {', '.join(clothing_types)}.

        Respond with 3 items separated by commas, no explanation.
        """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=False,
        temperature=0.7,
        top_p=0.9
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Get last line only (assumed to contain the answer)
    lines = decoded.strip().split("\n")
    
    last_line = lines[-1]
    # return last_line
    # Try to extract known clothing types
    suggestions = [x.strip().lower() for x in last_line.split(",")]

    return suggestions[:3]


In [55]:

# Example usage
user_query = "summer clothing"
top3 = get_top3_clothing_types(user_query, clothing_types, model, tokenizer)
print("Top 3 clothing types:", top3)

Top 3 clothing types: ['shirt', 'blouse', 'jacket']


In [52]:
print(top3)
last_line = top3
suggestions = [x.strip().lower() for x in top3.split(",")]
print(suggestions)

shirt, top, jacket
['shirt', 'top', 'jacket']
